# **Credit Card Fraud Detecction Assignment**

In [23]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import models,layers
from keras import regularizers

In [24]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
df = pd.read_csv('/content/drive/MyDrive/DL Assignments/creditcard.csv')
df.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [26]:
non_fraud=df[df["Class"]==0]
fraud = df[df["Class"]==1]
non_fraud=non_fraud.sample(3*fraud.shape[0])
data = fraud.append(non_fraud, ignore_index=True)

In [27]:
train_data = data.sample(frac= 0.7, random_state=125)
test_data =  data.drop(train_data.index)

In [28]:
train_labels = train_data.iloc[:,-1]
train_data = train_data.iloc[:,0:-1]
test_labels = test_data.iloc[:,-1]
test_data = test_data.iloc[:,0:-1]

In [29]:
train_mean = train_data.mean(axis=0)
train_data -= train_mean
train_std = train_data.std(axis=0)
train_data /= train_std
test_data -= train_mean
test_data /= train_std

In [30]:
train_data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
count,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03,1.378000e+03
mean,-9.668125e-18,-1.285458e-16,2.723189e-16,3.585263e-17,-8.524064e-17,-1.660903e-16,3.403180e-16,-3.192130e-16,4.342599e-17,-7.528750e-16,1.748722e-16,9.120265e-17,1.349509e-16,-8.878561e-17,5.847604e-16,-1.965852e-17,1.998683e-16,-2.475040e-16,1.096124e-16,1.053826e-16,-1.309225e-17,-4.355692e-17,1.804717e-17,-4.205634e-17,-2.207555e-17,-9.990396e-18,-8.218913e-17,-1.941682e-17,2.239782e-17,-1.901801e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.876007e+00,-6.444569e+00,-5.973838e+00,-5.833718e+00,-2.111306e+00,-8.235493e+00,-5.560683e+00,-8.811067e+00,-1.104741e+01,-6.343998e+00,-6.197009e+00,-1.672080e+00,-4.599043e+00,-3.235098e+00,-4.509388e+00,-4.611550e+00,-4.666472e+00,-4.685864e+00,-4.575750e+00,-3.632603e+00,-1.215265e+01,-1.073990e+01,-9.013068e+00,-1.903897e+01,-3.910735e+00,-7.865969e+00,-2.698170e+00,-1.107363e+01,-2.792215e+00,-3.409778e-01
25%,-8.447164e-01,-5.347840e-02,-4.274425e-01,-3.508004e-02,-6.348171e-01,-7.351042e-02,-4.284920e-01,4.274114e-02,-8.463894e-02,-3.046072e-01,1.837273e-02,-6.569008e-01,-1.426273e-01,-6.912055e-01,-2.626490e-02,-5.730330e-01,-5.445523e-02,1.701012e-01,-1.544411e-01,-5.705117e-01,-2.276816e-01,-1.716380e-01,-5.568643e-01,-1.673262e-01,-5.786205e-01,-5.648272e-01,-6.601270e-01,-1.273159e-01,-1.363497e-01,-3.320429e-01
50%,-1.517297e-01,2.073025e-01,-1.622188e-01,3.102550e-01,-2.617622e-01,1.987660e-01,-8.068216e-02,2.576175e-01,-3.143213e-03,1.835344e-01,3.136193e-01,-2.439674e-01,3.926750e-01,2.073135e-02,4.102569e-01,3.834772e-02,3.391867e-01,3.162774e-01,2.436136e-01,-8.618971e-02,-7.425034e-02,-5.141380e-02,8.634771e-03,1.897711e-03,8.992155e-02,4.178200e-02,-9.456332e-02,-1.546123e-02,-2.890165e-02,-2.809243e-01
75%,9.557249e-01,5.650033e-01,1.388769e-01,5.220940e-01,3.160380e-01,4.169520e-01,3.436839e-01,4.007080e-01,1.250383e-01,5.377227e-01,4.544817e-01,2.788119e-01,5.656569e-01,6.761370e-01,5.697083e-01,6.736366e-01,5.438821e-01,4.451661e-01,5.217653e-01,4.259361e-01,1.777497e-01,8.982347e-02,5.555962e-01,1.892444e-01,7.354231e-01,6.141111e-01,5.566854e-01,2.299683e-01,1.384198e-01,-4.299391e-02
max,1.694795e+00,8.212080e-01,6.724124e+00,1.083812e+00,3.989006e+00,4.156920e+00,1.203465e+01,6.896590e+00,5.340985e+00,3.374017e+00,3.295577e+00,4.657849e+00,1.280607e+00,3.125373e+00,1.397837e+00,2.750687e+00,1.734425e+00,1.791165e+00,2.261415e+00,4.361274e+00,9.400806e+00,1.266231e+01,8.449082e+00,5.450411e+00,5.031230e+00,3.586429e+00,5.679396e+00,5.333927e+00,2.322558e+01,2.109872e+01


In [31]:
train_data = train_data.to_numpy()
train_labels = train_labels.to_numpy().astype('float64')

test_data = test_data.to_numpy()
test_labels = test_labels.to_numpy().astype('float64')

In [32]:
model = models.Sequential()
model.add(layers.Dense(20, activation='relu',kernel_regularizer=regularizers.l2(0.001), input_shape=(train_data.shape[1],)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [33]:
model.compile(loss= 'binary_crossentropy', optimizer= 'rmsprop', metrics=['accuracy'])

In [34]:
model.fit(train_data, train_labels, epochs=70,validation_split = 0.3)

Epoch 1/70
31/31 [==============================] - 1s 9ms/step - loss: 1.1141 - accuracy: 0.4962 - val_loss: 0.7098 - val_accuracy: 0.6329
Epoch 2/70
31/31 [==============================] - 0s 3ms/step - loss: 0.9116 - accuracy: 0.5786 - val_loss: 0.6810 - val_accuracy: 0.8357
Epoch 3/70
31/31 [==============================] - 0s 3ms/step - loss: 0.8079 - accuracy: 0.6212 - val_loss: 0.6672 - val_accuracy: 0.8841
Epoch 4/70
31/31 [==============================] - 0s 3ms/step - loss: 0.7443 - accuracy: 0.6717 - val_loss: 0.6517 - val_accuracy: 0.8841
Epoch 5/70
31/31 [==============================] - 0s 3ms/step - loss: 0.7097 - accuracy: 0.7477 - val_loss: 0.6360 - val_accuracy: 0.8816
Epoch 6/70
31/31 [==============================] - 0s 4ms/step - loss: 0.6813 - accuracy: 0.7493 - val_loss: 0.6176 - val_accuracy: 0.8816
Epoch 7/70
31/31 [==============================] - 0s 3ms/step - loss: 0.6542 - accuracy: 0.7694 - val_loss: 0.5975 - val_accuracy: 0.8816
Epoch 8/70
31/31 [==

In [35]:
loss, accuracy = model.evaluate(test_data, test_labels)

19/19 [==============================] - 0s 1ms/step - loss: 0.2085 - accuracy: 0.9492


In [36]:
loss

0.20848117768764496

In [37]:
accuracy

0.9491525292396545

In [38]:
predictions = model.predict(test_data)

In [39]:
y_pred = (predictions > 0.5)

In [40]:
conf_matrix = tf.math.confusion_matrix(
    test_labels, y_pred, num_classes=2, weights=None, dtype=tf.dtypes.int32,
    name=None
)

In [41]:
conf_matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[445,   3],
       [ 27, 115]], dtype=int32)>

In [42]:
result = accuracy * 100
result

94.91525292396545